# Tiled merge

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py --style=tree --animation=movie

## Create two tensors

In [ ]:
# 1D 
X = 100

a1 = Tensor.fromRandom(["X"], [X], (0.2,), 9, seed=10)
a1.setName("A")

b1 = Tensor.fromRandom(["X"], [X], (0.2,), 9, seed=30)
b1.setName("B")

displayTensor(a1)
displayTensor(b1)

## Pairwise merge

Merge two tensors using the coordinates as the "key"

In [ ]:
z1 = Tensor(rank_ids=["X"],shape=[X]).setName("Z")

a_x = a1.getRoot()
b_x = b1.getRoot()
z_x = z1.getRoot()

canvas = createCanvas(a1, b1, z1)

for x, (z_ref, (ab, a_val, b_val)) in z_x << (a_x | b_x):
    # Merge a value from either A or B
    
    if ab == "A" or ab == "AB":
        z_ref <<= a_val
    else:
        z_ref <<= b_val
        
    addFrame(canvas, (x,), (x,), (x,))

displayTensor(z1)
displayCanvas(canvas)

## Merge after coordinate space slitting

In [ ]:
split = 4

z2 = Tensor(rank_ids=["X.1", "X.0"], shape=[(X+split-1)//split, split]).setName("Z2")

a2 = a1.splitUniform(split)
b2 = b1.splitUniform(split)

displayTensor(a2)

a_x1 = a2.getRoot()
b_x1 = b2.getRoot()
z_x1 = z2.getRoot()

canvas = createCanvas(a2, b2, z2)

for x1, (z_x0, (ab1, a_x0, b_x0)) in z_x1 << (a_x1 | b_x1):
    if ab1 == "A":
        # Merge an entire fiber from A
        
        for x0, (z_ref, a_val) in z_x0 << a_x0:
            z_ref <<= a_val

        # Broken - cannot replace a whole fiber...
        #position = z_x1.getPosition(x1)
        #z_x1[position] = a_x0        

        addFrame(canvas, (x1,), (), (x1,))
        
    elif ab1 == "B":
        # Merge an entire fiber from B
        
        for x0, (z_ref, b_val) in z_x0 << b_x0:
            z_ref <<= b_val

        # Broken - cannot replace a whole fiber...
        #position = z_x1.getPosition(x1)
        #z_x1[position] = b_x0
        
        addFrame(canvas, (), (x1,), (x1,))
    else:
        # Individually merge the elements from the rank 0 fibers of A and B
        
        for x0, (z_ref, (ab0, a_val, b_val)) in z_x0 << (a_x0 | b_x0):
            if ab0 == "A" or ab0 == "AB":
                z_ref <<= a_val
            else:
                z_ref <<= b_val

            addFrame(canvas, (x1,x0), (x1, x0), (x1, x0))
    
displayCanvas(canvas)

## Testing area

For running alternative algorithms